### Project milestones:
- Initial analysis of the data including overall statistics on user counts, geographies, discussion activity etc. (breakdown into markets)
- overall sentiment analysis of the data (breakdown into markets)
- identification of top 5 likes and dislikes (breakdown into markets)
- competitor comparison based on the top 5 likes and dislikes


In [1]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py', encoding="utf8").read())

demoji.download_codes()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.61 seconds)
Writing emoji data to C:\Users\chiso\.demoji\codes.json ...
... OK
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chiso\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chiso\AppData\Roaming

In [30]:
zalando = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_@zalando.json")
asos = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_asos.json")
bohoo = pd.read_json(r"..\Zalando_Analytics_HWR_BIPM\twitterdata\2021-03-05_en_bohoo.json")

In [9]:
zalando.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,1323291681899294721,133,GreenerRetail,Who and what has made our greener retailing tr...,0,Planet Earth,05-Mar-2021,70,False
1,1259761930304196610,1,fattybabycat,".@Zalando offers nothing more than scripted, c...",0,,05-Mar-2021,0,False
2,835467350249963521,2296,piotrkarwatka,We've got a great talk with Fabian Wesner on h...,0,"Wrocław, Polska",05-Mar-2021,1347,False
3,58485935,114,MissyDawn27586,@Zalando recieved wrong item... don’t know how...,0,,05-Mar-2021,27,False
4,1059339103928492033,11725,CarolCooney7,@DrewLawDesign @Zalando have sent me a few de...,0,,05-Mar-2021,457,False


In [10]:
asos.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,91800594,97473,_phelma,Quickly came to return my ASOS package &amp; t...,0,"London, England",05-Mar-2021,2070,False
1,78974281,5481,ZiyandaO,@ASOS_HeretoHelp why has my account been block...,0,"London, England",05-Mar-2021,821,False
2,15790423,229382,ASOS,@_justollie @PostOffice @RoyalMailHelp @RoyalM...,0,London,05-Mar-2021,1085110,True
3,1208987533192957953,7184,RHBlackMan,Birthday Tingz. #LockdownStyle #InstagramSawIt...,0,Barbados,05-Mar-2021,323,False
4,1216824142726561792,28,SAM92075802,ASOS I’ve been awaiting for a parcel since 15 ...,0,,05-Mar-2021,15,False


In [11]:
bohoo.head()

,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified
0,136420228,152284,MissNellyNels,@fragileceour Wahala for who no be Bohoo chic,0,,05-Mar-2021,5744,False
1,136420228,152284,MissNellyNels,Bohoo take all my money https://t.co/3a7hbF5OX5,0,,05-Mar-2021,5744,False
2,1354040830592335874,800,thepurpsoul,@EB0YGGUKZ @swagijoon Twitter bohoo👎😤,0,Bangtan's Heart,05-Mar-2021,45,False
3,930457106,172758,renethelover,@neeuQcMxelA @enctrI you clearly view the issu...,0,,05-Mar-2021,2662,False
4,1174750574333632512,10807,driphaver,"@240pee bohoo cry about it,,\nWHAT A PUTT TO T...",0,Jo#8762 | id/thrasher,05-Mar-2021,633,False


In [12]:
print('zalando size:', zalando.shape)
print('asos size:', asos.shape)
print('bohoo size:', bohoo.shape)

zalando size: (32, 9)
asos size: (3000, 9)
bohoo size: (72, 9)


In [2]:
df = pd.read_json(r"data_n_models\one_cleaned_df.json")

In [3]:
df.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
0,0,793418126500000000,521,MILESmobility,Good News! Alle interessierten MitarbeiterInne...,0,"Berlin, Deutschland",05-Mar-2021,768,False,de,@zalando,NaN,NaN
1,1,1119231287000000128,23,pici1303,@Zalando Ich weiß dass der Schein da rein muss...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
2,2,1119231287000000128,23,pici1303,@Zalando Ich hab mich nur gefragt wofür dieser...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
3,3,1119231287000000128,23,pici1303,@Zalando Ich hab doch schon den Rücksendeschei...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN
4,4,1119231287000000128,23,pici1303,@Zalando bei der Retoure ist noch ein einzelne...,0,,04-Mar-2021,0,False,de,@zalando,NaN,NaN


In [4]:
df["keyword"].value_counts()

asos        97402
boohoo      39731
zalando     32214
@zalando     3176
bohoo         846
Name: keyword, dtype: int64

In [5]:
df["language"].value_counts()

en    109987
fr     28384
es     18634
de     10605
it      4796
sv       745
fi       218
Name: language, dtype: int64

In [10]:
filters = ["zalando", "@zalando"]
zalando_en= df[(df["language"].isin(["en"]) & df["keyword"].isin(filters))]

In [13]:
print(zalando_en.shape)
zalando_en.head()


(10334, 14)


,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
383,0,1323291682000000000,133,GreenerRetail,Who and what has made our greener retailing tr...,0,Planet Earth,05-Mar-2021,70,False,en,@zalando,NaN,NaN
384,1,1259761930000000000,1,fattybabycat,".@Zalando offers nothing more than scripted, c...",0,,05-Mar-2021,0,False,en,@zalando,NaN,NaN
385,2,835467350200000128,2296,piotrkarwatka,We've got a great talk with Fabian Wesner on h...,0,"Wrocław, Polska",05-Mar-2021,1347,False,en,@zalando,NaN,NaN
386,3,58485935,114,MissyDawn27586,@Zalando recieved wrong item... don’t know how...,0,,05-Mar-2021,27,False,en,@zalando,NaN,NaN
387,4,1059339104000000000,11725,CarolCooney7,@DrewLawDesign @Zalando have sent me a few de...,0,,05-Mar-2021,457,False,en,@zalando,NaN,NaN


In [14]:
shuffled_df = zalando_en.sample(frac=1, random_state= 42)
shuffled_df.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN
140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18
15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18
63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,,11-Mar-2021,157,False,en,zalando,3.0,NaN
9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18


In [17]:
first200 = shuffled_df.iloc[:200, :]
first200.shape

(200, 14)

In [18]:
first200.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN
140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18
15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18
63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,,11-Mar-2021,157,False,en,zalando,3.0,NaN
9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18


## Manual Labelling comparison

In [38]:
first200.to_excel(r"data_n_models\first200.xlsx")

In [21]:
first200.head(20)

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id
121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN
140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18
15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18
63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,,11-Mar-2021,157,False,en,zalando,3.0,NaN
9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18
150614,272,1371600559000000000,17,scriptingtool,Zalando - Link Scraper\n - redesigned and impr...,0,,05-Apr-2021,82,False,en,zalando,4.0,1.374408e+18
3494,7,830115018900000000,3220,EmpowerWithTech,From BBC News to Director of Outfits at Zaland...,0,Leeds,05-Mar-2021,6300,False,en,zalando,NaN,NaN
28006,30,1345043875,34196,Functionalworks,Check out Zalando SE in Berlin\n\nThey're look...,0,London | New York,10-Mar-2021,7545,False,en,zalando,0.0,NaN
131292,347,1319976952000000000,66,fasthandsAIO,Since the 3.0 only Zalando orders arrived 😎 \n...,0,Deutschland,25-Mar-2021,5,False,en,zalando,1.0,NaN
92784,145,281244257,20179,CrisWorthington,Zalando laud’s society’s champions and celebra...,3,Vancouver Canada,23-Mar-2021,51923,False,en,zalando,0.0,NaN


In [44]:
first200.iloc[87]

index                                                                  353
user_id                                                1362305882000000000
user_statuses                                                           35
name                                                            BobProxies
tweet                    To Celebrate Todays INSANE Zalando Success,\n\...
retweets                                                                 1
location                                                                  
created                                                        06-Apr-2021
followers                                                              298
is_user_verified                                                     False
language                                                                en
keyword                                                            zalando
favorite_count                                                        14.0
in_reply_to_status_id    

In [54]:
labelled = pd.read_excel(r"data_n_models\first200.xlsx")

In [55]:
labelled.head()

,Column1,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,sentiments
0,121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN,0
1,140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18,0
2,15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18,0
3,63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,NaN,11-Mar-2021,157,False,en,zalando,3.0,NaN,2
4,9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,NaN,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18,0


In [56]:
labelled["cleaned_4_vader"] = clean_vader(labelled["tweet"])
labelled.head(1)

<string>:102: FutureWarning: The default value of regex will change from True to False in a future version.
<string>:122: FutureWarning: The default value of regex will change from True to False in a future version.


,Column1,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,sentiments,cleaned_4_vader
0,121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN,0,"covid interrupting our lives more, I’m trying ..."


In [57]:
#Vater sentiment analyzer
#The Compound score is a metric that calculates the sum of all the lexicon ratings which have been normalized between -1(most extreme negative) and +1 (most extreme positive).
sid_obj = SentimentIntensityAnalyzer()
labelled['compound'] = [sid_obj.polarity_scores(c)['compound'] for c in labelled['cleaned_4_vader']]
labelled.head()

,Column1,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,sentiments,cleaned_4_vader,compound
0,121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN,0,"covid interrupting our lives more, I’m trying ...",0.8090
1,140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18,0,Maybe you can help finally cop zalando,0.4019
2,15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18,0,have not had success zalando would like have f...,0.4770
3,63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,NaN,11-Mar-2021,157,False,en,zalando,3.0,NaN,2,Instagram post albareche Valencia🍊 streetitall...,0.0000
4,9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,NaN,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18,0,feel zalando getting activity then mesh recently,0.0000


In [58]:
labelled["compound"] = np.where(labelled["compound"] > 0, 4, labelled["compound"])      # 4 means positive
labelled["compound"] = np.where(labelled["compound"] == 0, 2, labelled["compound"])     # 2 means neutral
labelled["compound"] = np.where(labelled["compound"] < 0, 0, labelled["compound"])      # 0 means negative
labelled.head()

,Column1,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,language,keyword,favorite_count,in_reply_to_status_id,sentiments,cleaned_4_vader,compound
0,121416,84,963382625400000000,283,PrinNaidoo,"As covid is interrupting our lives more, I’m t...",0,"Berlin, Germany",29-Mar-2021,148,False,en,zalando,1.0,NaN,0,"covid interrupting our lives more, I’m trying ...",4.0
1,140876,305,1252027224000000000,1440,dreaze7,@NuevSolution Maybe you can help to finally co...,0,Belgique,30-Mar-2021,27,False,en,zalando,0.0,1.376913e+18,0,Maybe you can help finally cop zalando,4.0
2,15672,47,2873242068,1514,Yi_y827,@STARLABScripts I have not had success in zala...,0,BOTTER🤖,07-Mar-2021,14,False,en,zalando,0.0,1.368653e+18,0,have not had success zalando would like have f...,4.0
3,63743,472,1291338754000000000,2743,ARecheBrasil,📸| @_albxreche || Instagram post #albareche \n...,1,NaN,11-Mar-2021,157,False,en,zalando,3.0,NaN,2,Instagram post albareche Valencia🍊 streetitall...,2.0
4,9793,184,1268624052000000000,4293,krimlnalz,@NoCaptchaLoop i feel as if zalando is getting...,0,NaN,03-Mar-2021,40,False,en,zalando,0.0,1.367110e+18,0,feel zalando getting activity then mesh recently,2.0


In [59]:
labelled["sentiments"].value_counts()

2    91
0    69
4    40
Name: sentiments, dtype: int64

In [60]:
labelled["compound"].value_counts()

4.0    98
2.0    73
0.0    29
Name: compound, dtype: int64

In [63]:
y_test = labelled["sentiments"]
y_pred = labelled["compound"]
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.26      0.37        69
           2       0.58      0.46      0.51        91
           4       0.23      0.57      0.33        40

    accuracy                           0.41       200
   macro avg       0.48      0.43      0.40       200
weighted avg       0.52      0.41      0.43       200

